In [1]:
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#import doubletdetection
from scipy.stats import median_abs_deviation as mad
import numpy as np

In [3]:
os.environ['R_HOME'] = r"C:\Users\test\anaconda3\envs\CMD_mag\Lib\R"
os.environ['R_LIBS'] = r"C:\Users\test\anaconda3\envs\CMD_mag\Lib\R\library"

print("R_HOME:", os.environ['R_HOME'])
print("R_LIBS:", os.environ['R_LIBS'])

R_HOME: C:\Users\test\anaconda3\envs\CMD_mag\Lib\R
R_LIBS: C:\Users\test\anaconda3\envs\CMD_mag\Lib\R\library


In [4]:
import rpy2.robjects as ro
print(ro.r("R.version.string"))


[1] "R version 4.4.2 (2024-10-31 ucrt)"



In [5]:
%load_ext rpy2.ipython

C:\Users\test\anaconda3\envs\CMD_mag\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [6]:
%%R
print("Hello World")

[1] "Hello World"


In [10]:
import anndata2ri


In [8]:
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

In [9]:
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

C:\Users\test\AppData\Local\Temp\ipykernel_14864\1464119859.py:3: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [11]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp

Attaching package: 'SeuratObject'

The following objects are masked from 'package:base':

    intersect, t

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: 'MatrixGenerics'

The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts,

In [12]:
import warnings 
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [13]:
path_filtered_files_h5 = 'C:/Users/test/Documents/yohan/Data processing pipeline/scRNA/Data/h5_files_filtered'

In [14]:
adatas = [x for x in os.listdir(path_filtered_files_h5 ) 
          if x.endswith('.h5')]

In [15]:
def load_it(adata):
    samp= adata.split('_')[5]
    dpi_end = adata.split('_')[6]
    dpi = dpi_end.split('.')[0]
    adata = sc.read_10x_h5(path_filtered_files_h5 + '/' + adata)
    adata.obs['Sample'] = samp
    adata.obs['dpi'] = dpi
    adata.obs['Id'] = adata.obs['Sample'] + '_' + adata.obs['dpi']
    adata.obs.index = adata.obs.index + '-' + samp + '_' + dpi
    remove = ['Sample','dpi']

    adata.obs = adata.obs[[x for x in adata.obs.columns if x not in remove]]
    return adata

In [16]:
adatas = [load_it(ad) for ad in adatas]

In [17]:
adatas

[AnnData object with n_obs × n_vars = 11004 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 9137 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 10542 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 8525 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome']

In [18]:
for ad in adatas:
    ad.var_names_make_unique()

In [19]:
def qc(adata):
    sc.pp.filter_cells(adata, min_genes = 200)
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata.var["ribo"] = adata.var_names.str.startswith("RPS", "RPL")
    adata.var["hb"] = adata.var_names.str.startswith("^HB[^(P)]")
    sc.pp.calculate_qc_metrics(adata,qc_vars=["mt","ribo","hb"], inplace = True, percent_top = [20], log1p=True)

    remove = ['total_counts_mt', 'log1p_total_counts_mt', 'total_counts_ribo',
              'log1p_total_counts_ribo','total_counts_hb','log1p_total_counts_hb']

    adata.obs = adata.obs[[x for x in adata.obs.columns if x not in remove]]
    return adata

In [20]:
adatas = [qc(ad) for ad in adatas]

In [24]:
data_mat = [x.X.T.todense() for x in adatas]

In [25]:
#scDbFinder

In [ ]:
%%R -i data_mat -o doublet_scores_list -o doublet_classes_list

doublet_scores_list <- list()
doublet_classes_list <- list()


for (i in seq_along(data_mat)) {
  current_data <- data_mat[[i]]
  set.seed(123)
  sce <- scDblFinder(
    SingleCellExperiment(
      list(counts = current_data)
    )
  )
  doublet_scores_list[[i]] <- sce$scDblFinder.score
  doublet_classes_list[[i]] <- sce$scDblFinder.class
}

In [ ]:
for i,ad in enumerate(adatas):
    ad.obs["scDblFinder_score"] = doublet_scores_list[i]
    ad.obs["scDblFinder_class"] = doublet_classes_list[i]
    print(ad.obs['Id'][0])
    ad.obs.scDblFinder_class.value_counts()

In [ ]:
def pp(adata):
    adata = adata[adata.obs.pct_counts_mt < 6] 
    
    bool_vector = mad_outlier(adata, 'log1p_total_counts', 5) +\
            mad_outlier(adata, 'log1p_n_genes_by_counts', 5) +\
            mad_outlier(adata, 'pct_counts_in_top_20_genes', 5) +\
            mad_outlier(adata, 'pct_counts_mt', 3, upper_only = True)
    adata = adata[~bool_vector]

    adata.uns['cells_removed'] = sum(bool_vector)

    #doublets = clf.fit(adata.X).predict(p_thresh=1e-16, voter_thresh=0.6)
    #doublet_score = clf.doublet_score()

    #adata.obs["doublet_dbd"] = doublets
    #adata.obs["doublet_score_dbd"] = doublet_score

    adata.uns['doublets_removed_dbd'] = adata.obs.scDblFinder_class.sum()
    #adata = adata[adata.obs.doublet_dbd == 0]
    adata = adata[adata.obs.scDblFinder_class == 0]

    return adata

In [ ]:
adatas = [pp(ad) for ad in adatas]

In [ ]:
# partie avec choix de la prédiction en fonction d'une méthode machine learning ou pondérée

In [ ]:
for adata in adatas:
    print(len(adata), adata.uns['cells_removed'], adata.uns['doublets_removed_dbd'])

In [ ]:
os.makedirs('C:/Users/test/Documents/yohan/Data processing pipeline/scRNA/Output/processed_data', exist_ok = True)

In [ ]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join("processed_data", f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")